In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-07 11:16:43.998544: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 11:16:44.106944: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-07 11:16:44.106961: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-07 11:16:44.128801: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-07 11:16:44.662594: W tensorflow/stream_executor/platform/de

In [2]:
AUC_ROC=[]
model=[]
def list_auc(list_f,list_m):
    main_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


    external_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)
    
    read_m = pd.read_csv("main_set_plus_AtomPairFP.csv")
    df_m = pd.DataFrame(read_m)
    a1 = df_m['active']
    c1=[]
    for b1 in a1:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_m=pd.DataFrame()
    y_m.insert(loc=0, column='active', value=a1)
    y_m.insert(loc=1, column='inactive', value=c1)
    X_m=main_all_feature

    read_e = pd.read_csv("external_set_plus_AtomPairFP.csv")
    df_e = pd.DataFrame(read_e)
    a2 = df_e['active']
    c1=[]
    for b1 in a2:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_e=pd.DataFrame()
    y_e.insert(loc=0, column='active', value=a2)
    y_e.insert(loc=1, column='inactive', value=c1)
    X_e=external_all_feature

    batch_size = 8192
    num_classes = 2
    model = Sequential()
    model.add(Dense(800,
                    activation='relu',
                    input_dim=len(X_m.columns),
                    kernel_initializer = initializers.RandomNormal(stddev=0.02),
                    bias_initializer = initializers.RandomNormal(mean=1.),
                    kernel_regularizer = regularizers.l2(1e-1)))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt,
                  metrics=['AUC'])
    model.fit(X_m, y_m,
              batch_size=batch_size,
              epochs=100)
    score = model.evaluate(X_e, y_e, verbose=0)
    print(score[1])
    AUC_ROC.append(score[1])
    model=''

In [3]:
m='all_model'
a=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100


2022-11-07 11:16:49.071912: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 11:16:49.072459: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-07 11:16:49.072511: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-07 11:16:49.072551: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-07 11:16:49.072590: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

1/1 [==============================] - 0s 394ms/step - loss: 2.1820 - auc: 0.6269
Epoch 2/100
1/1 [==============================] - 0s 33ms/step - loss: 2.0516 - auc: 0.8341
Epoch 3/100
1/1 [==============================] - 0s 40ms/step - loss: 2.0161 - auc: 0.8578
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 1.8607 - auc: 0.8759
Epoch 5/100
1/1 [==============================] - 0s 35ms/step - loss: 1.6308 - auc: 0.8982
Epoch 6/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4773 - auc: 0.9146
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 1.4297 - auc: 0.9086
Epoch 8/100
1/1 [==============================] - 0s 36ms/step - loss: 1.3591 - auc: 0.9209
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 1.2806 - auc: 0.9374
Epoch 10/100
1/1 [==============================] - 0s 39ms/step - loss: 1.1823 - auc: 0.9609
Epoch 11/100
1/1 [==============================] - 0s 32ms/step - loss: 1.0981 

1/1 [==============================] - 0s 25ms/step - loss: 0.1851 - auc: 0.9949
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1806 - auc: 0.9960
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1829 - auc: 0.9954
Epoch 91/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1803 - auc: 0.9957
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1814 - auc: 0.9952
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1830 - auc: 0.9948
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1801 - auc: 0.9954
Epoch 95/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1790 - auc: 0.9959
Epoch 96/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1822 - auc: 0.9950
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1833 - auc: 0.9954
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [4]:
m='remove_AtomPairFP'
a=['Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 345ms/step - loss: 2.9889 - auc: 0.2725
Epoch 2/100
1/1 [==============================] - 0s 35ms/step - loss: 1.8205 - auc: 0.7481
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7517 - auc: 0.8583
Epoch 4/100
1/1 [==============================] - 0s 37ms/step - loss: 1.8621 - auc: 0.8795
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.9205 - auc: 0.8871
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 1.9343 - auc: 0.8853
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 1.8249 - auc: 0.8978
Epoch 8/100
1/1 [==============================] - 0s 31ms/step - loss: 1.7133 - auc: 0.9045
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 1.5436 - auc: 0.9160
Epoch 10/100
1/1 [==============================] - 0s 33ms/step - loss: 1.3596 - auc: 0.9294
Epoch 11/100
1/1 [==============================] - 0s 41ms/step - l

1/1 [==============================] - 0s 30ms/step - loss: 0.2193 - auc: 0.9940
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2183 - auc: 0.9947
Epoch 90/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2178 - auc: 0.9946
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2204 - auc: 0.9932
Epoch 92/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2192 - auc: 0.9936
Epoch 93/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2199 - auc: 0.9928
Epoch 94/100
1/1 [==============================] - 0s 32ms/step - loss: 0.2149 - auc: 0.9945
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2148 - auc: 0.9943
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2191 - auc: 0.9930
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2137 - auc: 0.9943
Epoch 98/100
1/1 [==============================] - 0s 31ms/step - loss: 

In [5]:
m='remove_Connectivity'
a=['AtomPairFP','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 345ms/step - loss: 2.5272 - auc: 0.3580
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 1.8460 - auc: 0.7906
Epoch 3/100
1/1 [==============================] - 0s 39ms/step - loss: 1.9127 - auc: 0.8442
Epoch 4/100
1/1 [==============================] - 0s 37ms/step - loss: 1.9130 - auc: 0.8712
Epoch 5/100
1/1 [==============================] - 0s 36ms/step - loss: 1.8938 - auc: 0.8742
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.7760 - auc: 0.8807
Epoch 7/100
1/1 [==============================] - 0s 31ms/step - loss: 1.5981 - auc: 0.8981
Epoch 8/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3944 - auc: 0.9165
Epoch 9/100
1/1 [==============================] - 0s 29ms/step - loss: 1.2894 - auc: 0.9234
Epoch 10/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2159 - auc: 0.9316
Epoch 11/100
1/1 [==============================] - 0s 34ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.1897 - auc: 0.9960
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1966 - auc: 0.9954
Epoch 90/100
1/1 [==============================] - 0s 32ms/step - loss: 0.1943 - auc: 0.9959
Epoch 91/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1985 - auc: 0.9941
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1911 - auc: 0.9955
Epoch 93/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1946 - auc: 0.9951
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1824 - auc: 0.9966
Epoch 95/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1959 - auc: 0.9942
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1942 - auc: 0.9952
Epoch 97/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1931 - auc: 0.9951
Epoch 98/100
1/1 [==============================] - 0s 30ms/step - loss: 

In [6]:
m='remove_Charge'
a=['AtomPairFP','Connectivity',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 338ms/step - loss: 1.9666 - auc: 0.7033
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 1.9929 - auc: 0.8244
Epoch 3/100
1/1 [==============================] - 0s 31ms/step - loss: 1.7991 - auc: 0.8518
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 1.5911 - auc: 0.8662
Epoch 5/100
1/1 [==============================] - 0s 35ms/step - loss: 1.4541 - auc: 0.8749
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 1.3482 - auc: 0.8913
Epoch 7/100
1/1 [==============================] - 0s 38ms/step - loss: 1.2941 - auc: 0.8991
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 1.1586 - auc: 0.9442
Epoch 9/100
1/1 [==============================] - 0s 37ms/step - loss: 1.0648 - auc: 0.9660
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 0.9999 - auc: 0.9754
Epoch 11/100
1/1 [==============================] - 0s 32ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.1836 - auc: 0.9957
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1789 - auc: 0.9961
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1855 - auc: 0.9943
Epoch 91/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1776 - auc: 0.9955
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1835 - auc: 0.9955
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1807 - auc: 0.9955
Epoch 94/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1782 - auc: 0.9957
Epoch 95/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1839 - auc: 0.9950
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1732 - auc: 0.9967
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1842 - auc: 0.9947
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [7]:
m='remove_Kappa'
a=['AtomPairFP','Connectivity',
       'Charge','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 471ms/step - loss: 1.9362 - auc: 0.7696
Epoch 2/100
1/1 [==============================] - 0s 31ms/step - loss: 1.7517 - auc: 0.7658
Epoch 3/100
1/1 [==============================] - 0s 40ms/step - loss: 1.5660 - auc: 0.8542
Epoch 4/100
1/1 [==============================] - 0s 36ms/step - loss: 1.4484 - auc: 0.8959
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.3087 - auc: 0.9275
Epoch 6/100
1/1 [==============================] - 0s 48ms/step - loss: 1.2151 - auc: 0.9451
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 1.1338 - auc: 0.9592
Epoch 8/100
1/1 [==============================] - 0s 33ms/step - loss: 1.0738 - auc: 0.9662
Epoch 9/100
1/1 [==============================] - 0s 38ms/step - loss: 0.9891 - auc: 0.9794
Epoch 10/100
1/1 [==============================] - 0s 31ms/step - loss: 0.9287 - auc: 0.9848
Epoch 11/100
1/1 [==============================] - 0s 37ms/step - l

1/1 [==============================] - 0s 34ms/step - loss: 0.1803 - auc: 0.9953
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1800 - auc: 0.9946
Epoch 90/100
1/1 [==============================] - 0s 44ms/step - loss: 0.1853 - auc: 0.9947
Epoch 91/100
1/1 [==============================] - 0s 35ms/step - loss: 0.1845 - auc: 0.9946
Epoch 92/100
1/1 [==============================] - 0s 38ms/step - loss: 0.1752 - auc: 0.9961
Epoch 93/100
1/1 [==============================] - 0s 35ms/step - loss: 0.1807 - auc: 0.9950
Epoch 94/100
1/1 [==============================] - 0s 32ms/step - loss: 0.1711 - auc: 0.9959
Epoch 95/100
1/1 [==============================] - 0s 36ms/step - loss: 0.1816 - auc: 0.9950
Epoch 96/100
1/1 [==============================] - 0s 35ms/step - loss: 0.1741 - auc: 0.9965
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1689 - auc: 0.9968
Epoch 98/100
1/1 [==============================] - 0s 34ms/step - loss: 

In [8]:
m='remove_MAP4'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MHFP6','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 354ms/step - loss: 1.9701 - auc: 0.7753
Epoch 2/100
1/1 [==============================] - 0s 34ms/step - loss: 1.7805 - auc: 0.7658
Epoch 3/100
1/1 [==============================] - 0s 31ms/step - loss: 1.6026 - auc: 0.8504
Epoch 4/100
1/1 [==============================] - 0s 38ms/step - loss: 1.5100 - auc: 0.8795
Epoch 5/100
1/1 [==============================] - 0s 35ms/step - loss: 1.3702 - auc: 0.9114
Epoch 6/100
1/1 [==============================] - 0s 41ms/step - loss: 1.2755 - auc: 0.9270
Epoch 7/100
1/1 [==============================] - 0s 38ms/step - loss: 1.1915 - auc: 0.9433
Epoch 8/100
1/1 [==============================] - 0s 40ms/step - loss: 1.1042 - auc: 0.9596
Epoch 9/100
1/1 [==============================] - 0s 40ms/step - loss: 1.0253 - auc: 0.9727
Epoch 10/100
1/1 [==============================] - 0s 43ms/step - loss: 0.9818 - auc: 0.9745
Epoch 11/100
1/1 [==============================] - 0s 36ms/step - l

1/1 [==============================] - 0s 39ms/step - loss: 0.1995 - auc: 0.9937
Epoch 89/100
1/1 [==============================] - 0s 34ms/step - loss: 0.1992 - auc: 0.9939
Epoch 90/100
1/1 [==============================] - 0s 40ms/step - loss: 0.2013 - auc: 0.9939
Epoch 91/100
1/1 [==============================] - 0s 40ms/step - loss: 0.1995 - auc: 0.9933
Epoch 92/100
1/1 [==============================] - 0s 39ms/step - loss: 0.1995 - auc: 0.9937
Epoch 93/100
1/1 [==============================] - 0s 34ms/step - loss: 0.2048 - auc: 0.9928
Epoch 94/100
1/1 [==============================] - 0s 42ms/step - loss: 0.2030 - auc: 0.9934
Epoch 95/100
1/1 [==============================] - 0s 42ms/step - loss: 0.2050 - auc: 0.9930
Epoch 96/100
1/1 [==============================] - 0s 35ms/step - loss: 0.2049 - auc: 0.9931
Epoch 97/100
1/1 [==============================] - 0s 40ms/step - loss: 0.1958 - auc: 0.9939
Epoch 98/100
1/1 [==============================] - 0s 36ms/step - loss: 

In [9]:
m='remove_MHFP6'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 378ms/step - loss: 3.1948 - auc: 0.2618
Epoch 2/100
1/1 [==============================] - 0s 32ms/step - loss: 1.8607 - auc: 0.6959
Epoch 3/100
1/1 [==============================] - 0s 33ms/step - loss: 1.6759 - auc: 0.8669
Epoch 4/100
1/1 [==============================] - 0s 38ms/step - loss: 1.8356 - auc: 0.8769
Epoch 5/100
1/1 [==============================] - 0s 36ms/step - loss: 1.9231 - auc: 0.8877
Epoch 6/100
1/1 [==============================] - 0s 37ms/step - loss: 1.9399 - auc: 0.8856
Epoch 7/100
1/1 [==============================] - 0s 33ms/step - loss: 1.8682 - auc: 0.8975
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 1.7887 - auc: 0.9015
Epoch 9/100
1/1 [==============================] - 0s 35ms/step - loss: 1.6512 - auc: 0.9098
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4941 - auc: 0.9202
Epoch 11/100
1/1 [==============================] - 0s 35ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.2427 - auc: 0.9906
Epoch 89/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2385 - auc: 0.9904
Epoch 90/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2359 - auc: 0.9920
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2340 - auc: 0.9910
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2342 - auc: 0.9912
Epoch 93/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2258 - auc: 0.9935
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2286 - auc: 0.9916
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2278 - auc: 0.9922
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2324 - auc: 0.9918
Epoch 97/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2256 - auc: 0.9920
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [10]:
m='remove_MOE'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MHFP6','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 342ms/step - loss: 2.8885 - auc: 0.2858
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.8493 - auc: 0.7580
Epoch 3/100
1/1 [==============================] - 0s 28ms/step - loss: 1.8271 - auc: 0.8551
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 1.9710 - auc: 0.8620
Epoch 5/100
1/1 [==============================] - 0s 36ms/step - loss: 1.9802 - auc: 0.8771
Epoch 6/100
1/1 [==============================] - 0s 36ms/step - loss: 1.9129 - auc: 0.8856
Epoch 7/100
1/1 [==============================] - 0s 29ms/step - loss: 1.8188 - auc: 0.8922
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6554 - auc: 0.9018
Epoch 9/100
1/1 [==============================] - 0s 27ms/step - loss: 1.4786 - auc: 0.9153
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3245 - auc: 0.9275
Epoch 11/100
1/1 [==============================] - 0s 31ms/step - l

1/1 [==============================] - 0s 25ms/step - loss: 0.2095 - auc: 0.9947
Epoch 89/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2088 - auc: 0.9942
Epoch 90/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2020 - auc: 0.9954
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2049 - auc: 0.9949
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2024 - auc: 0.9951
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2049 - auc: 0.9951
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2053 - auc: 0.9945
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2025 - auc: 0.9951
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2034 - auc: 0.9945
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2060 - auc: 0.9944
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [11]:
m='remove_MorganFP'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MHFP6','MOE','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 1s 635ms/step - loss: 1.9614 - auc: 0.6760
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 1.9390 - auc: 0.8265
Epoch 3/100
1/1 [==============================] - 0s 25ms/step - loss: 1.8355 - auc: 0.8503
Epoch 4/100
1/1 [==============================] - 0s 37ms/step - loss: 1.6080 - auc: 0.8744
Epoch 5/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4579 - auc: 0.8828
Epoch 6/100
1/1 [==============================] - 0s 35ms/step - loss: 1.3737 - auc: 0.8875
Epoch 7/100
1/1 [==============================] - 0s 36ms/step - loss: 1.3252 - auc: 0.8898
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.2190 - auc: 0.9242
Epoch 9/100
1/1 [==============================] - 0s 26ms/step - loss: 1.1243 - auc: 0.9497
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 1.0526 - auc: 0.9630
Epoch 11/100
1/1 [==============================] - 0s 33ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.1962 - auc: 0.9958
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2020 - auc: 0.9939
Epoch 90/100
1/1 [==============================] - 0s 34ms/step - loss: 0.1894 - auc: 0.9963
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1863 - auc: 0.9960
Epoch 92/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1944 - auc: 0.9958
Epoch 93/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1994 - auc: 0.9947
Epoch 94/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1985 - auc: 0.9949
Epoch 95/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2017 - auc: 0.9945
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1939 - auc: 0.9951
Epoch 97/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1919 - auc: 0.9959
Epoch 98/100
1/1 [==============================] - 0s 29ms/step - loss: 

In [12]:
m='remove_Topology'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MHFP6','MOE','MorganFP','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 340ms/step - loss: 2.7210 - auc: 0.3152
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.8212 - auc: 0.7660
Epoch 3/100
1/1 [==============================] - 0s 23ms/step - loss: 1.7826 - auc: 0.8644
Epoch 4/100
1/1 [==============================] - 0s 43ms/step - loss: 1.8946 - auc: 0.8794
Epoch 5/100
1/1 [==============================] - 0s 36ms/step - loss: 1.8956 - auc: 0.8883
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 1.8383 - auc: 0.8973
Epoch 7/100
1/1 [==============================] - 0s 27ms/step - loss: 1.7386 - auc: 0.9038
Epoch 8/100
1/1 [==============================] - 0s 34ms/step - loss: 1.5563 - auc: 0.9194
Epoch 9/100
1/1 [==============================] - 0s 25ms/step - loss: 1.4199 - auc: 0.9241
Epoch 10/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2680 - auc: 0.9385
Epoch 11/100
1/1 [==============================] - 0s 32ms/step - l

1/1 [==============================] - 0s 25ms/step - loss: 0.1912 - auc: 0.9960
Epoch 89/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1975 - auc: 0.9951
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1970 - auc: 0.9959
Epoch 91/100
1/1 [==============================] - 0s 32ms/step - loss: 0.1945 - auc: 0.9954
Epoch 92/100
1/1 [==============================] - 0s 34ms/step - loss: 0.1938 - auc: 0.9951
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2009 - auc: 0.9944
Epoch 94/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1931 - auc: 0.9952
Epoch 95/100
1/1 [==============================] - 0s 32ms/step - loss: 0.1919 - auc: 0.9953
Epoch 96/100
1/1 [==============================] - 0s 31ms/step - loss: 0.1993 - auc: 0.9942
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1969 - auc: 0.9951
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [13]:
m='remove_TorsionFP'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MHFP6','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 458ms/step - loss: 2.2667 - auc: 0.4818
Epoch 2/100
1/1 [==============================] - 0s 55ms/step - loss: 1.7819 - auc: 0.8248
Epoch 3/100
1/1 [==============================] - 0s 38ms/step - loss: 1.8731 - auc: 0.8541
Epoch 4/100
1/1 [==============================] - 0s 35ms/step - loss: 1.8463 - auc: 0.8738
Epoch 5/100
1/1 [==============================] - 0s 28ms/step - loss: 1.7501 - auc: 0.8817
Epoch 6/100
1/1 [==============================] - 0s 26ms/step - loss: 1.5864 - auc: 0.8996
Epoch 7/100
1/1 [==============================] - 0s 29ms/step - loss: 1.4381 - auc: 0.9061
Epoch 8/100
1/1 [==============================] - 0s 31ms/step - loss: 1.3129 - auc: 0.9175
Epoch 9/100
1/1 [==============================] - 0s 38ms/step - loss: 1.2369 - auc: 0.9253
Epoch 10/100
1/1 [==============================] - 0s 48ms/step - loss: 1.2172 - auc: 0.9162
Epoch 11/100
1/1 [==============================] - 0s 51ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.1964 - auc: 0.9953
Epoch 89/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1926 - auc: 0.9957
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1907 - auc: 0.9962
Epoch 91/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2038 - auc: 0.9939
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2030 - auc: 0.9943
Epoch 93/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1976 - auc: 0.9947
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1964 - auc: 0.9953
Epoch 95/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1944 - auc: 0.9958
Epoch 96/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2002 - auc: 0.9950
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2006 - auc: 0.9947
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [14]:
de=pd.DataFrame()
de['model']=model
de['auc']=AUC_ROC
de.to_csv('10feature_remove_one.csv',index=False)
print(de)

                  model       auc
0             all_model  0.945522
1     remove_AtomPairFP  0.934613
2   remove_Connectivity  0.944724
3         remove_Charge  0.943771
4          remove_Kappa  0.943724
5           remove_MAP4  0.941916
6          remove_MHFP6  0.947268
7            remove_MOE  0.945147
8       remove_MorganFP  0.942931
9       remove_Topology  0.944654
10     remove_TorsionFP  0.945857
